<a href="https://colab.research.google.com/github/jiyun-kong/outta_AI_bootcamp/blob/main/nlp_model_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

model

In [ ]:
!pip install transformers==3.0.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 769 kB 13.9 MB/s 
     |████████████████████████████████| 3.0 MB 31.7 MB/s 
     |████████████████████████████████| 880 kB 43.8 MB/s 
     |████████████████████████████████| 1.3 MB 38.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=e9cb33d735c5d056129509e0c5600b3656e09b45e8e7863054cc9dc8b11d3702
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [ ]:
def mount_drive():  
    from google.colab import drive
    drive.mount('/content/gdrive')

In [ ]:
if __name__ == "__main__":
    mount_drive()

Mounted at /content/gdrive


In [ ]:
import torch
from transformers import get_linear_schedule_with_warmup, BertForSequenceClassification, BertConfig

In [ ]:
from torch import nn
from transformers import BertModel, BertConfig

class BertClassifier(nn.Module):

    # BertClassifier는 초대용량 AI 모델인 BertModel을 파인튜닝한 모델이다.
    # 구체적으로, BertModel 위에 linear한 층 하나를 추가해 놓은 모델이다.
    # Bert -> Linear -> Softmax의 순서로 연산이 이루어진다.
    # 우리가 Loss를 구하는데 사용되는 CrossEntropyLoss의 경우, softmax를 자동으로 포함하여 연산해준다.
    # 고로, BertClassifier 클래스 내의 forward 함수에, softmax는 포함시키지 않는다.
    # 단, 학습의 용도가 아니라 테스트나 실제 시연을 할 때는 forward 결과물에 softmax를 적용시켜줘야 한다.

    def __init__(self, num_labels=7):

        super(BertClassifier, self).__init__()

        model_config = BertConfig.from_pretrained('monologg/kobert')
        
        # model_config와 from_pretrained 함수를 활용해서 bert 모델을 선언해주자.
        # self.bert = ~~ 꼴로 작성해주자.
        ## 여기에 코드 작성
        self.bert = model_config
        # nn 라이브러리를 활용해서 linear 모델을 선언해주자.
        # self.linear = ~~ 꼴로 작성해주자.
        # bert의 경우, 768차원 벡터를 산출해준다.
        # linear는 768차원의 값을 입력받아 7차원의 값을 출력해주어야 한다.
        ## 여기에 코드 작성
        self.linear = nn.Linear(in_features = 768, out_features = 7, bias = False)

    def forward(self, input_ids, attention_mask):
        # 위에서 생성한 self.bert, self.linear 두 개의 layer를 활용하면 된다.
        # bert -> linear를 적용해준 다음, 출력값을 리턴해주자.
        # Hint: 만약 __init__에서 self.bert라는 bert layer를 생성하였고 이를 forward 함수에서 활용하려면, 생성한 bert layer의 forward 함수를 활용하면 될 것이다.
        #       bert layer의 forward 함수는 input_ids와 attention_mask를 변수로 입력받아야 한다. 이에 따라, self.bert(input_ids, attention_mask)와 같은 방식으로 코드를 작성하면 될 것이다.
        
        ## 여기에 코드 작성
        outputs, outputs_cls, attn_probs = self.bert(input_ids, attention_mask)
        logits_cls = self.linear(outputs_cls)

        return logits_cls, attn_probs

In [ ]:
# 분류를 위한 BERT 모델 생성
# BertModel을 초기화하는 역할
def BertModelInitialization():
    PATH = "/content/gdrive/MyDrive/NLP/model.pt"
    
    # BertModel은 다양한 작업을 진행할 수 있도록 해주는 여러 인터페이스들을 제공한다.
    # 그 중, 본 중간 미션의 task에 가장 적합한 인터페이스를 찾아보자.
    # Hint 1 : HuggingFace 홈페이지에 Bert를 검색해서 찾아보자.
    # URL : https://huggingface.co/docs/transformers/main/en/index
    # Hint 2 : 본 중간 미션이, 영화리뷰를 긍정과 부정의 두 가지 감정으로 분류해내는 작업임을 고려해보자.

    model = BertClassifier()

    # 생성한 모델을 특정 PATH에 저장
    torch.save(model.state_dict(), PATH)

In [ ]:
def get_model(device, cuda_available):
    PATH = "model.pt"

    model = BertClassifier()

    if cuda_available:
        # PATH에 저장된 모델 불러옴
        model.load_state_dict(torch.load(PATH), strict=False)

        # 불러온 모델을 device에 등록
        model = model.to(device)
    else:
        # PATH에 저장된 모델을 불러오기 및 불러온 모델을 device에 등록
        model.load_state_dict(torch.load(PATH, map_location=device), strict=False)
    
    return model

**신경망 성능 향상을 위한 다양한 툴**



> 최종미션을 위한 이론문서를 확인해보자.


- 여러분은 옵티마이저, 에포크 수, 스케줄러 종류를 본 task에 가장 적합한 것으로 선택하여 바꿀 수 있다.
- 정답은 없다. 최적의 모델을 선택하여 신경망의 성능을 높여보자!




In [ ]:
def get_model_with_params(num_data, device, cuda_available):
    model = get_model(device, cuda_available)

    # 옵티마이저 설정
    optimizer = torch.optim.AdamW(model.parameters(),
                      lr = 1e-5, # 학습률
                      eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값,
                    )
    # 전체 데이터가 총 몇 번 학습되는지
    epochs = 3

    # 총 훈련 스텝
    total_steps = num_data * epochs

    # 학습이 이루어짐에 따라 learning_rate을 감소시키기 위한 스케줄러
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0,
                                                num_training_steps = total_steps)
  
    # nn 라이브러리에서 crossentropy 형식으로 loss를 계산하는 모델을 찾아서 손실함수로 설정해주자.
    # criterion = ~~ 꼴로 작성해주자.
    ## 여기에 코드 작성
    criterion = nn.CrossEntropyLoss()
    
    return model, optimizer, scheduler, epochs, criterion


In [ ]:
def main():
    %cd /content/gdrive/MyDrive/NLP
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    BertModelInitialization()
    print(get_model_with_params(34388, device, torch.cuda.is_available()))

In [ ]:
if __name__ == '__main__':
      main()

/content/gdrive/MyDrive/NLP


Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

(BertClassifier(
  (linear): Linear(in_features=768, out_features=7, bias=False)
), AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    initial_lr: 1e-05
    lr: 1e-05
    maximize: False
    weight_decay: 0.01
), <torch.optim.lr_scheduler.LambdaLR object at 0x7f9a9cbc90d0>, 3, CrossEntropyLoss())
